# Faire des statistiques

## Données

Mariannig Le Béchec - Aline Bouchard - Philippe Charrier - Claire Denecker - Gabriel Gallezot - Stéphanie Rennes

- Rapport : https://hal.archives-ouvertes.fr/hal-03545512v1
- Données : https://zenodo.org/record/5827206


In [2]:
import pandas as pd
df = pd.read_csv("https://zenodo.org/records/5827206/files/SOSP_Export_base%20de%20donn%C3%A9es%20diffusable.csv")

Deux variables :
- année de naissance
- présentation dans les réseaux sociaux

In [21]:
df["Formats_presentation_resultats_recherche_reseaux_sociaux"].unique()

array(['rarement', 'parfois', 'jamais', 'souvent', 'toujours'],
      dtype=object)

In [68]:
reco = {'rarement':1, 'parfois':2, 'jamais':0, 'souvent':3, 'toujours':4}
df["reseaux_q"] = df["Valorisation_recherche_reseaux_sociaux_num_generalistes"]
df["reseaux_n"] = df["Valorisation_recherche_reseaux_sociaux_num_generalistes"].replace(reco)

In [26]:
df["annee_q"] = pd.cut(df["annee_premiere_publi"], 4)
df["annee_n"] = df["annee_premiere_publi"]

## Univarié

Indicateurs

In [27]:
df["annee_premiere_publi"].mean()

2004.030303030303

Distribution

In [69]:
df["reseaux_q"].value_counts()

reseaux_q
jamais      741
parfois     106
rarement    106
souvent      77
toujours     59
Name: count, dtype: int64

recodage

## Bivarié

Quanti/quanti

In [70]:
df[["annee_n","reseaux_n"]].corr()

,annee_n,reseaux_n
annee_n,1.000000,0.192851
reseaux_n,0.192851,1.000000


In [71]:
from scipy.stats import pearsonr
pearsonr(df["annee_n"],df["reseaux_n"])

PearsonRResult(statistic=0.19285061858630462, pvalue=1.3890221497996923e-10)

Quanti/quali

In [72]:
df.groupby("reseaux_n")["annee_n"].mean()

reseaux_n
0    2002.731444
1    2004.415094
2    2006.839623
3    2007.142857
4    2010.542373
Name: annee_n, dtype: float64

In [73]:
a = list(df.groupby("reseaux_n")["annee_n"].apply(list))

In [74]:
from scipy.stats import f_oneway
f_oneway(*a)

F_onewayResult(statistic=10.73068474537501, pvalue=1.5652446052537652e-08)

Possibilité d'utiliser d'autres bibliothèques, comme pinguin https://pingouin-stats.org/build/html/index.html

In [57]:
#pip install pingouin

In [60]:
import pingouin as pg
aov = pg.anova(data=df, dv='annee_n', between='reseaux_n', detailed=True)
aov

,Source,SS,DF,MS,F,p-unc,np2
0,reseaux_n,961.069233,4,240.267308,1.866877,0.11399,0.006842
1,Within,139510.930767,1084,128.700121,NaN,NaN,NaN


Quali/Quali

In [37]:
pd.crosstab(df["reseaux_q"],df["annee_q"], normalize="columns")

annee_q,"(1967.948, 1981.0]","(1981.0, 1994.0]","(1994.0, 2007.0]","(2007.0, 2020.0]"
reseaux_q,,,,
jamais,0.695652,0.702326,0.648718,0.631236
parfois,0.086957,0.083721,0.100000,0.130152
rarement,0.217391,0.158140,0.174359,0.156182
souvent,0.000000,0.046512,0.058974,0.056399
toujours,0.000000,0.009302,0.017949,0.026030


In [39]:
from scipy.stats import chi2_contingency

In [40]:
t = pd.crosstab(df["reseaux_q"],df["annee_q"])
chi2_contingency(t)

Chi2ContingencyResult(statistic=9.980359089452108, pvalue=0.6176838187449825, dof=12, expected_freq=array([[ 15.01652893, 140.37190083, 254.62809917, 300.98347107],
       [  2.51331497,  23.49403122,  42.61707989,  50.37557392],
       [  3.7805326 ,  35.33976125,  64.1046832 ,  75.77502296],
       [  1.24609734,  11.64830119,  21.12947658,  24.97612489],
       [  0.44352617,   4.14600551,   7.52066116,   8.88980716]]))

In [35]:
#?pd.crosstab

## Intermède sur les tableaux

In [63]:
c = ['Valorisation_recherche_referencement',
 'Valorisation_recherche_reseaux_sociaux_num_generalistes',
 'Valorisation_recherche_reseaux_sociaux_num_academiques',
 'Valorisation_recherche_presence_num',
 'Valorisation_recherche_multimédia',
 'Valorisation_recherche_media_ligne',
 'Valorisation_recherche_sciences_citoyennes',
 'Valorisation_recherche_transfert_technologie',
 'Valorisation_recherche_conferences',
 'Valorisation_recherche_hackatons']

In [67]:
t0 = df[c[0]].value_counts()
t1 = df[c[0]].value_counts()
pd.concat([t0,t1],axis=1)

,count,count
Valorisation_recherche_referencement,,
jamais,478,478
souvent,191,191
toujours,178,178
parfois,142,142
rarement,100,100


## Modèles

#### Régression logistique

Quels sont les prédicteurs du fait de diffuser sa recherche sur les médias sociaux

Création d'une variable 0 (pas du tout ou rarement) et 1 sinon

In [83]:
reco = {'rarement':0, 'parfois':1, 'jamais':0, 'souvent':0, 'toujours':0}
df["reseaux_b"] = df["Valorisation_recherche_reseaux_sociaux_num_generalistes"].replace(reco)
df["reseaux_b"].value_counts()

reseaux_b
0    983
1    106
Name: count, dtype: int64

In [112]:
df["sexe_r"] = df["sexe"].replace({"je ne souhaite pas répondre":None})

Avec Scikit-learn

In [75]:
from sklearn.linear_model import LogisticRegression

In [113]:
X = pd.get_dummies(df[["annee_premiere_publi","sexe_r"]], drop_first=True)
y = df["reseaux_b"]

In [114]:
modele = LogisticRegression()
modele.fit(X, y)
modele.coef_

array([[-1.11056683e-03, -1.86439035e-07]])

Avec Statsmodels

In [118]:
import statsmodels.api as sm
from patsy import dmatrices

In [119]:
y,X = dmatrices('reseaux_b ~ sexe_r + annee_n', 
                data = df, 
                return_type = 'dataframe')

In [126]:
modele = sm.Logit(y,X)
resultat = modele.fit()
resultat.summary()

Optimization terminated successfully.
         Current function value: 0.316656
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              reseaux_b   No. Observations:                 1045
Model:                          Logit   Df Residuals:                     1042
Method:                           MLE   Df Model:                            2
Date:                Fri, 03 May 2024   Pseudo R-squ.:                0.009814
Time:                        15:39:03   Log-Likelihood:                -330.91
converged:                       True   LL-Null:                       -334.19
Covariance Type:            nonrobust   LLR p-value:                   0.03764
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -48.9593     19.256     -2.543      0.011     -86.700     -11.219
sexe_r[T.une femme]     0.1321      0.209      0.631      0.528      -0.278       0.542
annee_n                 0.0233      0.010      2.425      0.015       0.004       0.042
=======================================================================================
"""

Avec la version toute intégrée

In [129]:
#pip install pyshs

In [130]:
import pyshs

In [131]:
pyshs.regression_logistique(df, "reseaux_b", ["sexe_r", "annee_n"])

OR       p            IC 95%
Variable   Modalité                                 
.Intercept             0.0  0.011*  0.00 [0.00-0.00]
annee_n    numérique  1.02  0.015*  1.02 [1.00-1.04]
sexe_r     un homme    ref                          
           une femme  1.14   0.528  1.14 [0.76-1.72]